In [ ]:
%matplotlib inline
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation, patches, rc
import ipywidgets as widgets
import matplotlib as mpl
rc('animation', html='html5')
rc('savefig', dpi=80)
rc('figure', dpi=80)
from IPython.display import YouTubeVideo, HTML, Audio

# Sistemas para el procesamiento de señales

## Definición de sistema

- *Análisis de señales:* El estudio de las señales y sus propiedades en el dominio del tiempo y frecuencia
- *Procesamiento de señales:* El diseño de **sistemas** que procesan **señales de entrada** y producen **señales de salida**
    - Adicionalmente, un sistema puede tener parámetros (entradas númericas o booleanas)
    - Adicionalmente, un sistema puede tener retornos (salidas númericas o booleanas)
    - Sistema sin señal de entrada: Oscilador
    - Sistema sin señal de salida: Detector/clasificador de señal
    - Existen sistemas analógicos (continuos) y digitales (discretos), nos enfocaremos en los últimos
    
   
<center><img src="../images/system.png"></center>

- Usaremos $x[n]$ para denotar la señal (discreta) de entrada y $X[k]$ su espectro
- Usaremos $y[n]$ para denotar la señal (discreta) de salida e $Y[k]$ su espectro
 


### Ejemplos de sistemas

1. Un sistema para reducir el ruido de una EEG

<center><img src="../images/system-denoise-eeg.png"></center>

1. Un sistema para mejorar (sharpen) una imagen fuera de foco

<center><img src="../images/system-sharpen.jpg"></center>

1. Un sistema para eliminar el eco de un audio

<center><img src="../images/system-echo.png"></center>


## Sistemas  sin memoria

Los sistemas sin memoria son de forma

$$
y[n] = f(x[n]),
$$

es decir la salida del sistema en un instante dado depende solo de la entrada en ese instante

### Ejemplos


- Sistema amplificador ideal 
$$
y[n] = A x[n], 
$$
donde $A>0$ se llama *ganancia*
    - provoca atenuación de la entrada si $0<A<1$
    - es un sistema identidad si $A=1$

- Sistema con corrupción cuadrática
$$
y_n = x_n + \epsilon x_n^2
$$

- Sistema saturador (clamp)
$$
y_n = \begin{cases} B &x_n > B \\x_n & x_n \in [-B, B]\\ -B & x_n < -B\end{cases}
$$
- Sistema rectificador
$$
y_n = | x_n |
$$

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
Fs = 22050; n = np.arange(0, 2, step=1.0/Fs)
x = 0.2*np.sin(2.0*np.pi*200*n); 
y = np.abs(x) #+ 2*x**2
ax.plot(n, x, label='input'); ax.plot(n, y, label='output'); 
ax.set_xlim([0, 0.1]); plt.legend()
Audio(clean_4_audio(y), rate=Fs)

## Sistema Lineal

Propiedades de los sistemas lineales

- **Homogeneidad:** Un cambio en la amplitud de la entrada produce un cambio equivalente en la salida

$$
f(cx[n]) = c f(x[n]) = c y[n]
$$

- **Aditividad:** Señales que se suman en la entrada producen señales que se suman en la salida

$$
f(x_1[n] + x_2[n]) = f(x_1[n]) + f(x_2[n]) = y_1[n] + y_2[n]
$$

    - Las señales pasan por el sistema sin interactuar entre ellas
- Si no se cumple alguna de estas propiedades el sistema es **no lineal**
- ¿Son los sistemas anteriores lineales?



### Otras propiedades de los sistemas lineales


- Una cascada de sistemas lineales forman un sistema lineal equivalente
    - **Conmutatividad:** El orden de los sistemas en la cascada no es relevante
<img src="../images/system-conmu.png" width="400px">

- **Principio de superposición**: 
    - Considere una método de descomposición de funciones (Fourier, Taylor, PCA)
    - Si descomponemos una señal en $M$ componentes: $x[n] = x_1[n] + x_2[n] + \ldots +  x_M[n]$
    - Y aplicamos un **sistema lineal** a cada componente $y_j[n] = f(x_j[n])$
    - Podemos recuperar la salida total usando **aditividad** como $y_1[n] + y_2[n] + \ldots +  y_M[n] = y[n]$

<img src="../images/system-superpos.png" width="400px">
    


## Sistemas con memoria

Un sistema con memoria es aquel cuya salida puede depender de 
- la entrada actual
- las entradas anteriores
- las salidas anteriores

$$
\begin{align}
y[n] = f(x[n], &x[n-1], x[n-2], \ldots, x[0], \\ \nonumber
&y[n-1], y[n-2], \ldots, y[0]) \nonumber
\end{align}
$$

esto también se conoce como **sistema causal**

Un **sistema no-causal** usa entradas futuras ($x[n+1]$, $x[n+2]$, ...) y por ende solo se puede implementar de forma offline (una vez que sea ha observado toda la señal)

### Ejemplos de sistemas lineales con memoria simples

- Sistema con un retardo (delay)

$$
y[n] = x[n-m],
$$
    - depende solo de "una" entrada anterior
    - el valor de m define que tan "antigua" es la entrada pasada

- Sistema reverberador (eco)

$$
y[n] = x[n] + A x[n-m],
$$

    - depende de una entrada "pasada" y la entrada actual
    - la ganancia controla si el "eco" se atenua o amplifica

- El delay no afecta la amplitud de los componentes frecuenciales pero si su fase
- Más adelante veremos que este es un tipo de filtro conocido como pasa-todo (*all-pass*)

In [ ]:
%%capture

fig, ax = plt.subplots(3, figsize=(6, 6), tight_layout=True)
n = np.arange(0, 400, step=1)
x = lambda m: np.sin(2.0*np.pi*0.05*(n-m)) 
f = sfft.fftshift(sfft.fftfreq(d=1, n=len(n)))

def update(m):
    m = m*0.5 # Para hacer la animación + fluida
    ax[0].cla(); ax[0].plot(n, x(m));
    X = sfft.fftshift(sfft.fft(x(m)))
    Xm = np.absolute(X); Xp = np.angle(X)
    # Espectro de magnitud:
    ax[1].cla(); ax[1].plot(f, Xm); 
    # Espectro de fase enmascarado con el espectro de magnitud
    ax[2].cla(); ax[2].plot(f, Xm*Xp/np.amax(Xm)); 
    ax[2].set_ylim([-np.pi, np.pi])
    angle_delay = Xp[np.argmax(Xm)]
    ax[2].set_title("%0.4f [rad], %0.0f [deg]" % (angle_delay, 180*angle_delay/np.pi))
    return ()

anim = animation.FuncAnimation(fig, update, frames=40, interval=100, blit=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3), tight_layout=True)
Fs=22050; n = np.arange(0, 4, step=1.0/Fs) 
x = lambda m: np.sin(2.0*np.pi*880*(n-m))*np.exp(-(n-m)**2/0.5**2)*np.heaviside(n-m, 0)
y = x(0)  #+ 0.5*x(1.)   + 0.25*x(2.) + 0.125*x(3.)
ax.plot(n, y);
Audio(y, rate=Fs, normalize=False)

- El eco en cambio si modifica el espectro de magnitud
- Notemos el efecto de interferencia constructiva y destructiva al modificar el retardo
- La señal retardada cancela la original cuando tiene la misma amplitud ($A=1$)

In [ ]:
%%capture
fig, ax = plt.subplots(3, figsize=(6, 6), tight_layout=True)
n = np.arange(0, 400, step=1)
x = lambda m: np.sin(2.0*np.pi*0.05*(n-m)) 
f = sfft.fftshift(sfft.fftfreq(d=1, n=len(n)))
A = 1.
def update(m):
    m = 0.5*m; y = x(0) + A*x(m)
    ax[0].cla(); ax[0].plot(n, x(0), n, A*x(m))
    ax[1].cla(); ax[1].plot(n, y); ax[1].set_ylim([-A-1.1, A+1.1])
    X = sfft.fftshift(sfft.fft(y))
    ax[2].cla(); ax[2].plot(f, np.absolute(X)); 
    ax[2].set_ylim([-20, (1+A)*len(n)/2 + 20])
#interact(update, m=IntSlider_nice(min=0, max=20), 
#         A=SelectionSlider_nice(options=[0.5, 1., 2.], value=1));
anim = animation.FuncAnimation(fig, update, frames=40, interval=100, blit=False)

In [ ]:
HTML(anim.to_html5_video())

Notemos que en el caso $A=1$

$$
\begin{align}
y[n] &= \sin(2\pi k_0 n) + \sin(2\pi k_0 (n-m)) \nonumber \\ 
&= 2 \cos(\pi k_0 m) \sin(2\pi k_0 (n-m/2))  \nonumber
\end{align}
$$

se anula si

$$
\begin{align}
\pi k_0 m & = \frac{\pi}{2} + \pi i, \quad i \in \mathbb{Z} \nonumber \\
m &= \frac{1 + 2i}{2 k_0}, \quad i \in \mathbb{Z} \nonumber
\end{align}
$$

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3), tight_layout=True)
n = np.arange(0, 4, step=1.0/22050)
x = lambda m: np.sin(2.0*np.pi*880*(n-m))*np.exp(-(n-m)**2/0.5**2)*np.heaviside(n-m, 0)
y = x(0) + 1.*x(1./(2*880))
ax.plot(n, y);
Audio(y, rate=22050, normalize=False)

Ejemplo de interferencia

https://www.youtube.com/watch?v=IU8xeJlJ0mk

## Sistema FIR 


Generalizando el ejemplo de sistema lineal reverberante a más retardos llegamos a 

$$
\begin{align}
y[n] &= h[0] x[n] + h[1] x[n-1] + h[2] x[n-2] + \ldots + h[L] x[n-L] \nonumber \\
&= \sum_{j=0}^{L} h[j] x[n-j] \nonumber \\
&= (h* x)[n] \nonumber 
\end{align}
$$

que se puede modelar como una convolución discreta o en pseuco-código como un ciclo iterativo
```
   y[n] = 0
   for j in 0 to L
       y[n] = y[n] + h[j] x[n-j]
```
y se conoce como


- sistema FIR (finite impulse response)
- sistema MA (moving average)
- sistema todo-zeros 


y es de orden L (posee L+1 coeficientes)

- ¿Es este sistema lineal?
- ¿Que ocurre si entra al sistema un impulso unitario?

### Intepretación como media movil (MA)

- El sistema FIR es equivalente a una media movil ponderada (*weighted moving average*)
- Los coeficientes del sistema son los ponderadores 

Por ejemplo sea un sistema de 3 coeficientes unitarios
$$
\begin{align}
y[n] = (h*x)[n] &= \sum_{j=0}^{2} h[j] x[n-j] \nonumber \\
&= x[n] + x[n-1] + x[n-2] \nonumber
\end{align}
$$
donde cada salida se calcula a partir de 
$$
\overbrace{x[0], x[1], x[2]}^{y[2]} , x[3], x[4], \ldots
$$
$$
x[0], \overbrace{x[1], x[2] , x[3]}^{y[3]}, x[4], \ldots
$$
$$
x[0], x[1], \overbrace{x[2] , x[3], x[4]}^{y[4]}, \ldots
$$

En este caso para obtener el valor de $y[0]$ e $y[1]$ se deben establecer "condiciones de borde"

### Ejemplo: Eliminando ruido blanco aditivo

In [ ]:
fig, ax = plt.subplots(3, figsize=(9, 7), tight_layout=True)
np.random.seed(0); 
n = np.arange(0, 100, step=1)
C = 5*np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/10**2)
x_clean = np.random.multivariate_normal(np.zeros_like(n), C) 
ax[0].plot(n, x_clean)
x = x_clean + 2*np.random.randn(len(n))
ax[0].plot(n, x, 'k.'); ylims = ax[0].get_ylim()
# System:
L = 10; h = np.ones(shape=(L,))/L; 
# Acumulator
y = np.zeros_like(n, dtype=np.float)

def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    return ()

def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+L] = h
    ax[1].cla(); ax[1].plot(n, c); 
    y[m] = np.sum(h*x[m:m+L])
    ax[2].cla(); ax[2].plot(n, y);  ax[2].set_ylim(ylims)
    ax[2].plot([m, m], [ylims[0], ylims[1]], 'r--', alpha=0.5)
    return ()

anim = animation.FuncAnimation(fig, update,init_func=init, 
                               frames=100-len(h), interval=200, blit=True)

En la práctica podemos usar [`scipy.signal.convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html)

In [ ]:
fig, ax = plt.subplots(3, figsize=(8, 6), tight_layout=True)
ax[0].plot(n, x_clean); ax[0].plot(n, x, 'k.')
L = 10; 
h = np.ones(shape=(L,)); 
#h = scipy.signal.tukey(L)
#h = scipy.signal.hamming(L)
h = h/np.sum(h)
ax[1].plot(h)
ax[2].plot(scipy.signal.convolve(x, h, mode='same', method='auto')); 
ax[2].set_ylim(ylims);

### Ejemplo: Encontrando cambios en una señal

In [ ]:
fig, ax = plt.subplots(3, figsize=(8, 6), tight_layout=True)
n = np.arange(0, 100, step=1)
x = np.zeros_like(n, dtype=np.float)
x[20:] += 1.; x[40:] += 1.; x[80:] -= 1.;
ax[0].plot(n, x)
# System:
h = np.array([-0.5, 0.5])
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    return ()

def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(n, c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y);  
    ax[2].plot([m, m], [-0.5, 0.5], 'r--', alpha=0.5)
    return ()

anim = animation.FuncAnimation(fig, update, init_func=init, frames=100-len(h), interval=200, blit=True)

### Ejemplo: Removiendo una tendencia

In [ ]:
fig, ax = plt.subplots(3, figsize=(8, 6), tight_layout=True)
np.random.seed(0); 
n = np.arange(0, 100, step=1)
C = np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/30**2)
x = np.sin(2.0*np.pi*0.1*n) + 5*np.random.multivariate_normal(np.zeros_like(n), C)
ax[0].plot(n, x); ylims = ax[0].get_ylim()
# System:
L = 5; h = -np.ones(shape=(L,))/L; h[L//2] += 1
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    return ()
    
def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(n, c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y);  
    ax[2].plot([m, m], [-0.5, 0.5], 'r--', alpha=0.5)
    return ()
    
anim = animation.FuncAnimation(fig, update, init_func=init, 
                               frames=100-len(h), interval=200, blit=True)

### Ejemplo: Detectando una patrón específico en una señal ruidosa

In [ ]:
def sine_pulse(n, k0, center):
    x = np.zeros_like(n, dtype=np.float); L = int(1/k0)
    x[center:center+L] = np.sin(2.0*np.pi*k0*(n[center:center+L] - n[center]))
    return x

fig, ax = plt.subplots(3, figsize=(8, 6), tight_layout=True)
np.random.seed(0); 
n = np.arange(0, 100, step=1)
x_clean = sine_pulse(n, k0=0.05, center=50) 
ax[0].plot(n, x_clean)
s = 1.;  x = x_clean + s*np.random.randn(len(n))
ax[0].plot(n, x, 'k.'); ylims = ax[0].get_ylim()
ax[0].set_title("SNR %0.2f [dB]" %(10*np.log(0.5/s**2)))
# System:
h = np.sin(2.0*np.pi*0.05*n[:int(len(n)/5)] + np.pi); h = h/np.sum(np.absolute(h))
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    return ()
    
def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y**2);  #ax[2].set_ylim(ylims)
    ax[2].plot([m, m], [np.amin(y**2), np.amax(y**2)], 'r--', alpha=0.5)
    return ()

anim = animation.FuncAnimation(fig, update, init_func=init, 
                               frames=100-len(h), interval=200, blit=True)

## Sistema invariante al desplazamiento

Un sistema es invariante al desplazamiento  (*shift-invariant*) de su entrada si 

$$
f(x[n-m]) = y[n-m] 
$$

Es decir que 
- Aplicar un retardo en la entrada provoca un retardo equivalente en la salida
    - Un *blip* en la entrada produce un *blop* en la salida sin importar **cuando** ocurre el blip
- Las características del sistema no cambian con $n$
- Si $n$ representa el tiempo decimos que el sistema es **invariante en el tiempo**


***
Un sistema FIR es invariante al desplazamiento si los valores de $h$ son constantes para todo $n$

$$
h[n] = \text{Cte} \quad \forall n
$$

***

In [ ]:
fig, ax = plt.subplots(2, figsize=(8, 5), tight_layout=True)
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); 
ax[0].set_ylim([-0.1, 1.1]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n));

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)      
    ax[1].set_ylim([-0.1, 1.1]); ax[1].set_title('Output')
    L = 50; h = np.ones(shape=(L, ))
    # h = 1.+ np.cos(2.0*np.pi*2.*(n-m)/len(n))
    h = h/np.sum(np.absolute(h))
    l0[0].set_ydata(x); l1[0].set_ydata(scipy.signal.convolve(x, h, mode='same'))
    return ()

anim = animation.FuncAnimation(fig, update, frames=500, interval=200, blit=True);

## Respuesta al impulso de un sistema


Sea el impulso unitario o delta de Kronecker
$$
\delta[n-m] = \begin{cases} 1 & n=m \\ 0 & n \neq m \end{cases}
$$

La **respuesta al impulso de un sistema discreto** es la salida obtenida cuando la entrada es un impulso unitario

***

Para un sistema FIR arbitrario tenemos

$$
y[n]|_{x=\delta} = (h * \delta)[n] = \sum_{j=0}^L h[j] \delta[n-j] = \begin{cases} h[n] & n \in [0, L] \\ 0 & \text{en otro caso} \end{cases} \\
$$

es decir que la respuesta al impulso:
-  tiene **una duración finita y luego decae a zero**
-  recupera los coeficientes $h[j]$ del sistema

En un sistema causal se tiene que $h[n] = 0 \quad \forall n < 0$

Llamamos **soporte** del sistema a todos aquellos valores de $n$ tal que  $h[n] \neq 0$

***

**Ejemplo:** Para 
$$
y[n] = x[n] + A x[n-m]
$$
la respuesta al impulso es: 
$$
y[n] = \delta[n] + A \delta[n-m] = \begin{cases} 1 & n=0\\ A& n=m \\ 0 & \text{en otro caso} \end{cases}
$$

## Sistema LTI y respuesta en frecuencia de un sistema

Los sistemas LTI (*linear time-invariant*) cumplen 

- linealidad: homogeneo y aditivo
- invarianza al desplazamiento de la entrada

y se expresan como

$$
y[n] = (h * x)[n] = \sum_j h[j] x[n-j]
$$
- h es la respuesta al impulso
- h guarda los coeficientes del sistema

***
Por propiedad de la DFT sabemos que un sistema LTI cumple

$$
\begin{align}
\text{DFT}_N [y[n]] & = \text{DFT}_N [(h * x)[n]] \nonumber \\
\text{DFT}_N [y[n]] & = \text{DFT}_N [h[n]]  \text{DFT}_N [x[n]] \nonumber \\
Y[k] &= H[k] X[k] , \nonumber
\end{align}
$$

donde convertimos la convolución temporal en una multiplicación frecuencial


- Llamamos a $H[k]$ la **respuesta en frecuencia del sistema** 
- La respuesta en frecuencia es la transformada de Fourier de la respuesta al impulso

***

Los sistemas LTI tienen intepretación directa en el dominio del frecuencia

***

En la próxima lección usaremos sistemas LTI para diseñar **filtros**